In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import relevant libraries
import pandas as pd
import numpy as np
import os
import random
import pickle
import re
# set project folder directory
DATA_DIR = '/content/drive/MyDrive/w266-finalproj/data'
subfolders = [ f.path for f in os.scandir('/content/drive/MyDrive') if f.is_dir() ]
print(subfolders)
#dir_list = os.listdir("/content/drive/MyDrive/W266 Project/w266-finalproj/data/02_intermediate")

['/content/drive/MyDrive/Books', '/content/drive/MyDrive/PROSPECT', '/content/drive/MyDrive/Python ECA', '/content/drive/MyDrive/CoinbaseWalletBackups', '/content/drive/MyDrive/Contents', '/content/drive/MyDrive/w266-finalproj', '/content/drive/MyDrive/pop_lstm_attpop_lstm_att', '/content/drive/MyDrive/Colab Notebooks', '/content/drive/MyDrive/ai_tune']


In [ ]:
# Reading in language CSV 
data = pd.read_csv(f"{DATA_DIR}/02_intermediate/language-processed-data.csv")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data["genre"].replace({"country": "Country", "soul": "Soul", "jazz":"Jazz", 
                            "folk":"Folk", "pop":"Pop", "metal":"Metal", "rb":"R&B", 
                            "rock":"Rock", "rap":"Rap"}, inplace=True)
print(data.groupby(['genre']).size())

genre
Folk      3983
Jazz      4840
Metal     7833
Pop       8248
R&B       4335
Rap       9957
Rock     10022
Soul      4064
dtype: int64


In [ ]:
def sample_data(genre, sample_size):
    random.seed(69)
    data1 = data.copy()
    data1 = data1[data1['genre'] == genre].sample(sample_size)
    data1['lyrics'] = data1['lyrics'].astype(str)
    # data1['lyrics']= data1['lyrics'].apply(preprocessText)
    return data1

In [ ]:
# sample a subset of data
pop_data = sample_data('Pop', 6000)
pop_data.to_csv(f"{DATA_DIR}/02_intermediate/pop_sample.csv")
print(pop_data.shape)
print(pop_data.genre.unique())

(6000, 6)
['Metal']


In [ ]:
data

,artist,genre,title,lyrics,word_num,language
0,Iron Maiden,Metal,The Number of the Beast,"Woe to you, o'er Earth and Sea\nFor the Devil ...",303,en
1,Iron Maiden,Metal,Fear of the Dark,I am a man who walks alone\nAnd when I'm walki...,358,en
2,Iron Maiden,Metal,The Trooper,You'll take my life but I'll take yours too\nY...,242,en
3,Iron Maiden,Metal,Hallowed Be Thy Name,"I'm waiting in my cold cell, when the bell beg...",261,en
4,Iron Maiden,Metal,Run to the Hills,White man came across the sea\nHe brought us p...,199,en
...,...,...,...,...,...,...
53277,Donny Hathaway,Soul,"The Ghetto - Live @ Troubadour, Hollywood, CA.","Whoo, yeah\nMm-hmm\nYes\nThis is the Ghetto\nS...",262,en
53278,Donny Hathaway,Soul,What’s going on - live version,"Mother, mother\nThere's too many of you crying...",159,en
53279,Donny Hathaway,Soul,Hey girl - live version,Hey girl\nI've been watching you\nThe rapid be...,161,en
53280,Donny Hathaway,Soul,Make It Your Own,"Yesterday, you were mine\nNow, you're gone\nI ...",88,en


In [ ]:
import re
def process_lyric(data, size=16):
    """
      Function to process each line of lyric into sentences with tokenized word
      Inputs:
        data: pd.DataFrame()
        size: length of token
      Output:
        TBD
    """

    data = data.copy()
    processed_lyric_by_genre = pd.DataFrame(columns=['genre', 'processed_lyric', 'word_num','language'])
    for i, row in data.iterrows():
        lyric = row['lyrics']
        lyric = lyric.replace('\n', ' \n ')
        lyric = lyric.replace('!', ' ! ')
        lyric = lyric.replace('?', ' ? ')
        lyric = lyric.replace('#', ' ')
        lyric = re.findall(r'\S+|\n', lyric)
        # concatenating the word list to create a corpus
        processed_lyric_tokenzied = [i.lower() for i in lyric][:size]
        temp_data = {
            "genre": row['genre'],
            "lyric": row['lyrics'],
            "processed_lyric": processed_lyric_tokenzied,
            "word_num": row['word_num'],
            "language": row['language'],
        }
        processed_lyric_by_genre = processed_lyric_by_genre.append(temp_data, ignore_index=True)
    return processed_lyric_by_genre

In [ ]:

processed_data = pd.DataFrame(columns=['genre', 'processed_lyric', 'word_num','language'])
for genre in data['genre'].unique():
  sliced = data[data['genre'] == genre].sample(n=100, replace=False, random_state=5634)
  processed_df = process_lyric(sliced, size=16)
  processed_data = processed_data.append(processed_df)
processed_data

,genre,processed_lyric,word_num,language,lyric
0,Metal,"[down, fell, the, stars, as, they, splashed, i...",163,en,Down fell the stars as they splashed into the ...
1,Metal,"[yesterday, is, gone, forever, \n, no, turning...",232,en,Yesterday is gone forever\nNo turning back the...
2,Metal,"[crisis, feeds, the, lunacy, \n, all, fear, th...",176,en,Crisis feeds the lunacy\nAll fear the new mach...
3,Metal,"[lay, beside, me,, tell, me, what, they've, do...",395,en,"Lay beside me, tell me what they've done\nSpea..."
4,Metal,"[the, sky, was, clear, that, night, \n, we, we...",83,en,The sky was clear that night\nWe were alone\nA...
...,...,...,...,...,...
95,Soul,"[lay, it, down,, lay, it, down,, lay, it, down...",315,en,"Lay it down, lay it down, lay it down\nPut you..."
96,Soul,"[was, blind,, but, now, i, see, \n, \n, amazin...",241,en,"Was blind, but now I see\n\nAmazing, Amazing G..."
97,Soul,"[there, ain't, no, reason, for, us, sitting, d...",336,en,There ain't no reason for us sitting down\nTo ...
98,Soul,"[willow, weep, for, me, \n, willow, weep, for,...",127,en,Willow weep for me\nWillow weep for me\nBend y...


In [ ]:
processed_data = processed_data.reset_index(drop=True)
processed_data.to_csv(f"{DATA_DIR}/16_tokens_seeds.csv")

In [ ]:
# Define Hard Code Max Sequence Length
MAX_SEQUENCE_LENGTH = 16

# define some preprosessing functions
# training data processing
stopChars = [',','(',')','.','-','[',']','"']

# preprocessing the corpus by converting all letters to lowercase, 
# replacing blank lines with blank string and removing special characters
def preprocessText(text):
    processedText = text.lower()
    for char in stopChars:
        processedText = processedText.replace(char,'')
    return processedText

# tokenization 
def corpusToList(corpus):
    corpusList = [w for w in corpus.split(' ')] 
    corpusList = [i for i in corpusList if i] #removing empty strings from list
    return corpusList
    

def tokenize_data(data):
    data = data.copy()
    word_list = []
    for i, row in data.iterrows():
        lyric = row['lyrics']
        lyric = lyric.replace('\n', ' \n ')
        lyric = lyric.replace('!', ' ! ')
        lyric = lyric.replace('?', ' ? ')
        lyric = lyric.replace('#', ' ')
        lyric = re.findall(r'\S+|\n', lyric)
        word_list.append(lyric)
    # concatenating the word list to create a corpus
    DP_text = [j for i in word_list for j in i]
    print('corpus length:', len(DP_text))
    return(DP_text)

# Create a dictionary of characters, see the index of characters.
def dictionary_maker(words):
    char_to_int = dict((c, i) for i, c in enumerate(words))
    int_to_char = dict((i, c) for i, c in enumerate(words))
    char_to_int['UNK'] = len(int_to_char)
    int_to_char[len(int_to_char)] = 'UNK'
    return(char_to_int, int_to_char)


def make_sentences_and_next_chars(seq_length, DP_text, step):
    sentences = []
    next_chars = []

    # Create Target and sentences window
    for i in range(0, len(DP_text) - seq_length, step):
        # range from current index to sequence length charaters
        sentences.append(DP_text[i: i + seq_length])  
        next_chars.append(DP_text[i + seq_length]) # the next character
    
    sentences = np.array(sentences)
    next_chars = np.array(next_chars)
    return(sentences, next_chars)

In [ ]:
pop_data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
DP_pop = tokenize_data(pop_data)
pop_words = sorted(list(set(DP_pop)))
pop_char_to_int, pop_int_to_char = dictionary_maker(pop_words)


corpus length: 19287661


In [ ]:
pickle.dump(pop_char_to_int,

{'\n': 0,
 '!': 1,
 '"': 2,
 '""': 3,
 '""You': 4,
 '"\'A': 5,
 '"\'Bout': 6,
 '"\'Cause': 7,
 '"\'Disappear\'': 8,
 '"\'Ere': 9,
 '"\'Heartbreaker\'': 10,
 '"\'I-I': 11,
 '"\'It': 12,
 '"\'Til': 13,
 '"\'Turn': 14,
 '"\'What': 15,
 '"\'Ye,': 16,
 '"\'le\'': 17,
 '"\'spec': 18,
 '"\'til': 19,
 '"\'till': 20,
 '"(...)': 21,
 '"(Forever)': 22,
 '"(Hello)': 23,
 '"(Inhales)': 24,
 '"(Let\'s': 25,
 '"(Shhh': 26,
 '"(This': 27,
 '"(spoken)Yo,': 28,
 '")': 29,
 '"*Pfft*,': 30,
 '"*}': 31,
 '",': 32,
 '"-Flames,': 33,
 '".': 34,
 '"..': 35,
 '"...': 36,
 '"..."BABA-BABA-BABA': 37,
 '"..."WHAAAA': 38,
 '"...(garbled)': 39,
 '"...And': 40,
 '"...Going': 41,
 '"...My': 42,
 '"...Plenty': 43,
 '"...Pluto': 44,
 '"...Scorpio,': 45,
 '"...Where': 46,
 '"...You': 47,
 '"...You\'re': 48,
 '"...a': 49,
 '"...and': 50,
 '"...blessed': 51,
 '"...by': 52,
 '"...it\'s': 53,
 '"...really': 54,
 '"...what': 55,
 '"..led': 56,
 '"..stomp': 57,
 '"0': 58,
 '"1': 59,
 '"1,2': 60,
 '"1-2,': 61,
 '"1-2-3-4"': 62